In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

## The Data

In [2]:
data_path = "data/winequality-red.csv"
data = pd.read_csv(data_path)

data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


## Tracking experiments

``` shell
mlflow server \
    --backend-store-uri mlruns/ \
    --default-artifact-root mlruns/ \
    --host 0.0.0.0 \
    --port 5000
```

In [3]:
remote_server_uri = "http://127.0.0.1:5000"
mlflow.set_tracking_uri(remote_server_uri)

In [4]:
mlflow.tracking.get_tracking_uri()

'http://127.0.0.1:5000'

In [5]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2023/07/07 12:24:54 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/sairam/TAMLEP-mlflow/mlruns/755656959556408307', creation_time=1688712894156, experiment_id='755656959556408307', last_update_time=1688712894156, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [7]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

def load_data(data_path):
    data = pd.read_csv(data_path)
    
    train, test = train_test_split(data)
    
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    
    return train_x, train_y, test_x, test_y

def train(alpha=0.5, l1_ratio=0.5):
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    data_path = "data/winequality-red.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)
    
    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        
        predict_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predict_qualities)
        
        print("Elasticnet model (alpha={:f}, l1_ratio={:f}):".format(alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [8]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.8222428497595401
  MAE: 0.6278761410160693
  R2: 0.12678721972772666
Save to: /home/sairam/TAMLEP-mlflow/mlruns/755656959556408307/2e32f36906d7407a9293b2693e2e33d2/artifacts


In [9]:
train(0.3, 0.3)

Elasticnet model (alpha=0.300000, l1_ratio=0.300000):
  RMSE: 0.7942719184710977
  MAE: 0.6192843288488945
  R2: 0.18518636291164137
Save to: /home/sairam/TAMLEP-mlflow/mlruns/755656959556408307/a4105465f8b54c15b096c869b53b2a98/artifacts


In [10]:
train(0.8, 0.8)

Elasticnet model (alpha=0.800000, l1_ratio=0.800000):
  RMSE: 0.8598685637629909
  MAE: 0.6478991380829888
  R2: 0.045042561953817395
Save to: /home/sairam/TAMLEP-mlflow/mlruns/755656959556408307/3612d2247cbb46048e35c5728db8a92f/artifacts
